# Ex04

## create a connection to the database using the library sqlite3

In [1]:
import sqlite3
import pandas as pd

connection = sqlite3.connect('../data/logs.db')
cursor = connection.cursor()

cursor.execute("ATTACH DATABASE '../data/checking-logs.sqlite' AS logs")

connection.commit()

## using only one query for each of the groups, create two dataframes: test_results and control_results with the columns time and avg_diff and only two rows
## we still are not using the lab ’project1’


In [73]:
test_query = """
WITH user_times AS (
    SELECT
        t.uid,
        CASE
            WHEN strftime('%s', t.first_commit_ts) < strftime('%s', t.first_view_ts) THEN 'before'
            ELSE 'after'
        END AS time,
        (d.deadlines - strftime('%s', t.first_commit_ts)) / 3600.0 AS delta_hours 
    FROM test AS t
    JOIN deadlines AS d ON t.labname = d.labs
    WHERE t.labname != 'project1'
),
filtered_users AS (
    SELECT uid
    FROM user_times
    GROUP BY uid
    HAVING COUNT(DISTINCT time) = 2
)
SELECT
    ut.time,
    AVG(ut.delta_hours) AS avg_diff
FROM user_times ut
INNER JOIN filtered_users fu ON ut.uid = fu.uid
GROUP BY ut.time
;
"""

control_query = """
WITH user_times AS (
    SELECT
        t.uid,
        CASE
            WHEN strftime('%s', t.first_commit_ts) < strftime('%s', t.first_view_ts) THEN 'before'
            ELSE 'after'
        END AS time,
        (d.deadlines - strftime('%s', t.first_commit_ts)) / 3600.0 AS delta_hours 
    FROM control AS t
    JOIN deadlines AS d ON t.labname = d.labs
    WHERE t.labname != 'project1'
),
filtered_users AS (
    SELECT uid
    FROM user_times
    GROUP BY uid
    HAVING COUNT(DISTINCT time) = 2
)
SELECT
    ut.time,
    AVG(ut.delta_hours) AS avg_diff
FROM user_times ut
INNER JOIN filtered_users fu ON ut.uid = fu.uid
GROUP BY ut.time
;
"""
test_results = pd.read_sql(test_query, connection)
control_results = pd.read_sql(control_query, connection)


print("Test Results:")
print(test_results)
print("\nControl Results:")
print(control_results)

connection.close()

Test Results:
     time    avg_diff
0   after  105.229241
1  before   61.156632

Control Results:
     time    avg_diff
0   after  118.144571
1  before   99.901448


## have the answer: did the hypothesis turn out to be true and the page does affect the students’ behavior?

The results come back inconclusive as in both cases after has a higher avg delta than before. The deltas are even higher in the control meaning that on average assignments were submitted earlier before the page was introduced. therefor it could be concluded that the page does not make them submit earlier but only delays the submition. Making the Hypothesis false.